---
title: Time resolution effect (data)
---

## Process Wind data

In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
from space_analysis.utils.speasy import Variables
from discontinuitypy.datasets import IDsDataset
from discontinuitypy.utils.basic import resample

from datetime import timedelta

from discontinuitypy.missions import wind_mag_h4_rtn_meta, wind_plasma_k0_swe_meta

In [15]:
ts = timedelta(seconds=1 / 11)
tau = timedelta(seconds=60)

In [16]:
timerange = ["2016-01-01", "2016-06-29"]
provider = "archive/local"
mag_meta = wind_mag_h4_rtn_meta
plasma_meta = wind_plasma_k0_swe_meta

wind_mag_vars = Variables(
    timerange=timerange,
    **mag_meta.model_dump(),
    provider=provider,
)

wind_plasma_vars = Variables(
    timerange=timerange,
    **plasma_meta.model_dump(),
    provider=provider,
)

In [11]:
wind_mag_data = wind_mag_vars.to_polars()
wind_plasma_data = wind_plasma_vars.to_polars()

In [19]:
# for freq in [11, 5 , 2, 1, 0.5]:
for freq in [5]:
    ts = timedelta(seconds=1 / freq)

    wind_ids_dataset = (
        IDsDataset(
            mag_data=wind_mag_data.pipe(resample, every=ts),
            plasma_data=wind_plasma_data,
            tau=tau,
            ts=ts,
            mag_meta=mag_meta,
            plasma_meta=plasma_meta,
        )
        .find_events(return_best_fit=False)
        .update_events()
        .export(f"data/ts_effect/events.Wind.ts_{1/freq:.2f}s_tau_60s.arrow")
    )

15-May-24 18:09:44: UserWarning: Ray execution environment not yet initialized. Initializing...
To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()


2024-05-15 18:09:46,850	INFO worker.py:1724 -- Started a local Ray instance.
15-May-24 18:09:48: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 14:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=78514) 15-May-24 18:09:56: RuntimeWarning: overflow encountered in exp
(_deploy_ray_func pid=78514) 
(_deploy_ray_func pid=78514) 15-May-24 18:09:57: RuntimeWarning: overflow encountered in exp
(_deploy_ray_func pid=78514) 
(_deploy_ray_func pid=78514) 15-May-24 18:10:02: RuntimeWarning: overflow encountered in exp [repeated 11x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_deploy_ray_func pid=78514)  [repeated 11x across cluster]
(_deploy_ray_func pid=78514) 15-May-24 18:10:07: RuntimeWarning: overflow encountered in exp [repeated 10x across cluster]
(_deploy_ray_func pid=78514)  [repeated 10x across cluster]
(_deploy_ray_func pid=78517) 15-May-24 18:10:13: RuntimeWarning: overflow encountered in exp [repeated 5x across cluster]
(_deploy_ray_func pid=78517)  [repeated 5x across cluster]


ValueError: conflicting sizes for dimension 'v_dim': length 50 on <this-array> and length 3 on {'time': 'time', 'v_dim': 'v_dim'}